# Itinerary Basics

Basic report on stage itinerary - stage name, distance etc.

In [485]:
import notebookimport

In [486]:
import os
import sqlite3
import pandas as pd

In [487]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='portugal18.db'
    conn = sqlite3.connect(dbname)
    rally='Portugal'
    rebase = 'PAD'
    rebase = ''

In [488]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

def traceryparse(rules, root, modifiers=base_english):
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)
    
    return grammar.flatten(root)
    

## Rally Summary

In [489]:
def dbGetRallySummary(rally):
    q='''
    SELECT o.*, t.totalDistance FROM (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
    COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}" AND type='StageStart') AS o
    JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
    '''.format(rally=rally)
    
    rallydetails = pd.read_sql(q,conn)
    
    return rallydetails

In [490]:
rs = dbGetRallySummary(rally)
rs

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Portugal,2018-05-17,2018-05-20,Vodafone Rally de Portugal,PRT,Gravel,20,358.19,1583.47


In [491]:
rs.dtypes

country.name       object
startDate          object
finishDate         object
name               object
country.iso3       object
surfaces           object
numOfStages         int64
compDistanceKm    float64
totalDistance     float64
dtype: object

In [492]:
rules = {'origin': "#name# (#startDate# to #finishDate#) #stages#. #dist#. #surface#.",
         'stages': "runs over #numOfStages# competitive special stages",
         'dist': "The distance covered on the special stages is #compDistanceKm.round#km, with an overall rally distance of #totalDistance.round#km",
         'surface':"The special stage surface type is predominantly #surfaces#",
        }

In [493]:
rs['report'] = rs.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english, pytracery_logic]), axis=1)

In [494]:
rs[rs['country.name']==rally]['report'].iloc[0]

'Vodafone Rally de Portugal (2018-05-17 to 2018-05-20) runs over 20 competitive special stages. The distance covered on the special stages is 358.19km, with an overall rally distance of 1583.47km. The special stage surface type is predominantly Gravel.'

In [495]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)

In [496]:
README='''# Rally Report - {}

*This report is unofficial and is not associated in any way with the Fédération Internationale de l’Automobile (FIA) or WRC Promoter GmbH.*


{}
'''.format(rally, rs[rs['country.name']==rally]['report'].iloc[0])

with open('{}/README.md'.format(basepath), 'w') as out_file:
    out_file.write(README)

In [497]:
import inflect
p=inflect.engine()

In [498]:
rs['startDate']=pd.to_datetime(rs['startDate'])
rs['startDate'].loc[0].strftime("%A %d,  %B %Y")

'Thursday 17,  May 2018'

In [499]:
p.number_to_words(p.ordinal((int(rs['startDate'].loc[0].strftime("%d")))))

'seventeenth'

## Itinerary Items

In [500]:
def dbGetTimeControls(rally):

    q='''
    SELECT il.name AS date, itc.*, ce.timeZoneOffset,
         isc.itinerarySectionId, isc.name AS section, isc.`order`
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
    time_controls = pd.read_sql(q,conn)
    time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
    return time_controls

In [501]:
q='''
    SELECT *
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
xx =pd.read_sql(q,conn)
xx.columns

Index(['code', 'controlId', 'controlPenalties', 'distance', 'eventId',
       'firstCarDueDateTime', 'firstCarDueDateTimeLocal', 'location',
       'stageId', 'status', 'targetDuration', 'targetDurationMs',
       'timingPrecision', 'type', 'itineraryLegId',
       'itinerarySections.itinerarySectionId', 'roundingPolicy', 'categories',
       'clerkOfTheCourse', 'country.countryId', 'country.iso2', 'country.iso3',
       'country.name', 'countryId', 'eventId', 'finishDate', 'location',
       'mode', 'name', 'organiserUrl', 'slug', 'startDate', 'stewards',
       'surfaces', 'templateFilename', 'timeZoneId', 'timeZoneName',
       'timeZoneOffset', 'trackingEventId', 'itineraryLegId',
       'itinerarySectionId', 'name', 'order', 'itineraryId', 'itineraryLegId',
       'legDate', 'name', 'order', 'startListId', 'status'],
      dtype='object')

In [502]:
xx.head()

,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,...,itinerarySectionId,name,order,itineraryId,itineraryLegId,legDate,name,order,startListId,status
0,TC0,1922,All,NaN,31,2018-05-17T17:10:00,2018-05-17T18:10:00+01:00,Guimaraes,462,Completed,...,182,Section 1,1,78,85,2018-05-17,Thursday 17th May,1,137.0,Completed
1,TC1,1958,All,40.60,31,2018-05-17T17:56:00,2018-05-17T18:56:00+01:00,Lousada,462,Completed,...,182,Section 1,1,78,85,2018-05-17,Thursday 17th May,1,137.0,Completed
2,SS1,1959,None,3.36,31,2018-05-17T18:03:00,2018-05-17T19:03:00+01:00,Lousada (SSS),462,Completed,...,182,Section 1,1,78,85,2018-05-17,Thursday 17th May,1,137.0,Completed
3,TC1A,1920,Late,48.22,31,2018-05-17T19:00:00,2018-05-17T20:00:00+01:00,Parc Ferme IN,-1,Completed,...,182,Section 1,1,78,85,2018-05-17,Thursday 17th May,1,137.0,Completed
4,TC1B,1919,All,NaN,31,2018-05-18T06:45:00,2018-05-18T07:45:00+01:00,Parc Ferme OUT - Service IN,-1,Completed,...,183,Section 2,2,78,84,2018-05-18,Friday 18th May,2,138.0,Running


In [503]:
import datetime
t=datetime.time(abs(int(360/60)), 360 % 60)
dt = datetime.datetime.combine(datetime.date.today(), t)
dt.isoformat()

'2018-05-18T06:00:00'

In [504]:
t = datetime.datetime.combine(datetime.date.today(),datetime.time(0))
(t  + datetime.timedelta( minutes=-359)).isoformat()

'2018-05-17T18:01:00'

In [505]:
xx[:2][['timeZoneId','timeZoneOffset']]

,timeZoneId,timeZoneOffset
0,GMT Standard Time,60
1,GMT Standard Time,60


In [506]:
if __name__=='__main__':
    time_controls = dbGetTimeControls(rally)
    display(time_controls.head())

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,roundingPolicy,timeZoneOffset,itinerarySectionId,section,order
0,Thursday 17th May,TC0,1922,All,NaN,31,2018-05-17T17:10:00,2018-05-17 17:10:00,Guimaraes,462,...,NaN,Minute,TimeControl,85,182,NoRounding,60,182,Section 1,1
1,Thursday 17th May,TC1,1958,All,40.60,31,2018-05-17T17:56:00,2018-05-17 17:56:00,Lousada,462,...,2760000.0,Minute,TimeControl,85,182,NoRounding,60,182,Section 1,1
2,Thursday 17th May,SS1,1959,None,3.36,31,2018-05-17T18:03:00,2018-05-17 18:03:00,Lousada (SSS),462,...,420000.0,Minute,StageStart,85,182,RoundToClosestMinute,60,182,Section 1,1
3,Thursday 17th May,TC1A,1920,Late,48.22,31,2018-05-17T19:00:00,2018-05-17 19:00:00,Parc Ferme IN,-1,...,3420000.0,Minute,TimeControl,85,182,NoRounding,60,182,Section 1,1
4,Friday 18th May,TC1B,1919,All,NaN,31,2018-05-18T06:45:00,2018-05-18 06:45:00,Parc Ferme OUT - Service IN,-1,...,NaN,Minute,TimeControl,84,183,NoRounding,60,183,Section 2,2


In [507]:
#Check datetime type
time_controls['firstCarDueDateTime'] = pd.to_datetime(time_controls['firstCarDueDateTime'])

In [508]:
import datetime

def newtime(row):
    t=datetime.timedelta( minutes=row['timeZoneOffset'])
    return row['firstCarDueDateTime']+ t

time_controls['mylocaltime'] = time_controls.apply(lambda row: newtime(row),axis=1)

In [509]:
rules = {'origin': "#mylocaltime.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
        }

In [510]:
dategroups = time_controls.groupby('date',sort=False)
for key in dategroups.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=dategroups.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        g2 = grouped2.get_group(key2)
        l=len(g2[g2['code'].str.startswith('SS')])
        print('{} - {} special {}\n'.format(key2,p.number_to_words(l),p.plural_noun('stage',l)))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 17th May:

Section 1 - one special stage

	18:10:00 TC0 Guimaraes  [None]
	18:56:00 TC1 Lousada (40.6km) [00:46:00]
	19:03:00 SS1 Lousada (SSS) (3.36km) [00:07:00]
	20:00:00 TC1A Parc Ferme IN (48.22km) [00:57:00]


---

Friday 18th May:

Section 2 - three special stages

	07:45:00 TC1B Parc Ferme OUT - Service IN  [None]
	08:04:00 TC1C Service OUT  [00:19:00]
	09:12:00 TC2 Santa Luzia (76.37km) [01:08:00]
	09:15:00 SS2 Viana do Castelo 1 (26.73km) [00:03:00]
	10:17:00 TC3 Senande (20.28km) [01:02:00]
	10:20:00 SS3 Caminha 1 (18.11km) [00:03:00]
	10:50:00 TC4 N305 (7.01km) [00:30:00]
	10:53:00 SS4 Ponte de Lima 1 (27.54km) [00:03:00]
	12:44:00 TC4A Porto - Rua Saraiva de Carvalho (99.27km) [01:51:00]
	12:55:00 TC4B Porto - Calcada Vandoma (3.4km) [00:11:00]
	13:25:00 TC4C Regroup & Technical Zone IN (28.13km) [00:30:00]


Section 3 - five special stages

	13:40:00 TC4D Regroup OUT - Service IN  [00:15:00]
	14:14:00 TC4E Service OUT  [00:34:00]
	15:22:00 TC5 Santa Luzia (7

In [511]:


def initStageReports(rebase='overall'):
    
    sectionREADME_base = '''### {section} Report
    '''

    SUMMARY ='\n'
    
    rules['stages'] = "#code# - #location# #distance.isNotNull(post=km).brackets#"

    dn = '' if not rebase or 'overall' in rebase else '_'+rebase
  
    sections = time_controls.groupby('section',sort=False)
    keyorder = [k for k in time_controls.sort_values('order')['section'].unique() if k in sections.groups.keys()]
    for key in keyorder:

        sectionfn = '{bp}/{key}_report{dn}.md'.format(bp=basepath,key=key, dn=dn)

        with open(sectionfn, 'w') as out_file:
            out_file.write('')

        sectionControls = sections.get_group(key)
        sstages = sectionControls[sectionControls['code'].str.startswith('SS')]
        l=len(sstages)
        title = '# {}, {}\n\nThis section comprises {} special {}'.format(key,sectionControls['date'].iloc[0], p.number_to_words(l),p.plural_noun('stage',l))
        sectionREADME = '''{title}'''.format( title=title)

        sstage=[]
        for r in sstages.apply(lambda row: pandas_row_mapper(row, rules, "#stages#",[base_english,inflect_english]), axis=1):
            sstage.append(r)
        sectionREADME = '{} ({})'.format(sectionREADME,', '.join(sstage))


        sectionREADME = '''{s}\n\nThe full scheduled itinerary for the section was as follows:\n'''.format(s=sectionREADME,)

        controls = []
        for r in sectionControls.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            controls.append(r)
        sectionREADME = '{}\n\t- {}\n'.format(sectionREADME, '\n\t- '.join(controls))

        print(sectionREADME,'\n------\n')
        with open(sectionfn, 'a') as out_file:
            out_file.write(sectionREADME)

        #Add section
        SUMMARY = '{summary}\n* [{key}]({key}_report{dn}.md)\n'.format(summary=SUMMARY, key=key,dn=dn)

        sstageDict = sstages.set_index('code').to_dict(orient='index')
        #Add special stages
        for s in sstageDict: #[Section 2](Section 2_report.md)
            SUMMARY = '{summary}  - [{s} - {n}]({s}_report{dn}.md)\n'.format(summary=SUMMARY,
                                                                              s=s, 
                                                                              n=sstageDict[s]['location'],
                                                                              dn=dn)

    with open('{}/SUMMARY{}.md'.format(basepath,dn), 'a') as out_file:
        out_file.write(SUMMARY)

initStageReports(rebase)
#initStageReports()

# Section 1, Thursday 17th May

This section comprises one special stage (SS1 - Lousada (SSS) (3.36km))

The full scheduled itinerary for the section was as follows:

	- 18:10:00 TC0 Guimaraes  [None]
	- 18:56:00 TC1 Lousada (40.6km) [00:46:00]
	- 19:03:00 SS1 Lousada (SSS) (3.36km) [00:07:00]
	- 20:00:00 TC1A Parc Ferme IN (48.22km) [00:57:00]
 
------

# Section 2, Friday 18th May

This section comprises three special stages (SS2 - Viana do Castelo 1 (26.73km), SS3 - Caminha 1 (18.11km), SS4 - Ponte de Lima 1 (27.54km))

The full scheduled itinerary for the section was as follows:

	- 07:45:00 TC1B Parc Ferme OUT - Service IN  [None]
	- 08:04:00 TC1C Service OUT  [00:19:00]
	- 09:12:00 TC2 Santa Luzia (76.37km) [01:08:00]
	- 09:15:00 SS2 Viana do Castelo 1 (26.73km) [00:03:00]
	- 10:17:00 TC3 Senande (20.28km) [01:02:00]
	- 10:20:00 SS3 Caminha 1 (18.11km) [00:03:00]
	- 10:50:00 TC4 N305 (7.01km) [00:30:00]
	- 10:53:00 SS4 Ponte de Lima 1 (27.54km) [00:03:00]
	- 12:44:00 TC4A Porto -

In [512]:
sstages['date']

61    Sunday 20th May
Name: date, dtype: object

In [513]:
sstages.set_index('code').to_dict(orient='index')

{'SS20': {'date': 'Sunday 20th May',
  'controlId': 1969,
  'controlPenalties': 'None',
  'distance': 11.18,
  'eventId': 31,
  'firstCarDueDateTime': Timestamp('2018-05-20 11:18:00'),
  'firstCarDueDateTimeLocal': Timestamp('2018-05-20 11:18:00'),
  'location': 'Fafe 2 (Power Stage)',
  'stageId': 464,
  'status': 'ToRun',
  'targetDuration': '00:03:00',
  'targetDurationMs': 180000.0,
  'timingPrecision': 'Minute',
  'type': 'StageStart',
  'itineraryLegId': 82,
  'itinerarySections.itinerarySectionId': 188,
  'roundingPolicy': 'RoundToClosestMinute',
  'timeZoneOffset': 60,
  'itinerarySectionId': 188,
  'section': 'Section 7',
  'order': 7,
  'mylocaltime': Timestamp('2018-05-20 12:18:00')}}

In [514]:
print(SUMMARY)

In [515]:
!head -n100 "report/Section 3_report.md"

# Section 3, Friday 18th May

This section comprises five special stages (SS5 - Viana do Castelo 2 (26.73km), SS6 - Caminha 2 (18.11km), SS7 - Ponte de Lima 2 (27.54km), SS8 - Porto Street Stage1 (1.95km), SS9 - Porto Street Stage2 (1.95km))

The full scheduled itinerary for the section was as follows:

	- 13:40:00 TC4D Regroup OUT - Service IN  [00:15:00]
	- 14:14:00 TC4E Service OUT  [00:34:00]
	- 15:22:00 TC5 Santa Luzia (76.37km) [01:08:00]
	- 15:25:00 SS5 Viana do Castelo 2 (26.73km) [00:03:00]
	- 16:27:00 TC6 Senande (20.28km) [01:02:00]
	- 16:30:00 SS6 Caminha 2 (18.11km) [00:03:00]
	- 17:00:00 TC7 N305 (7.01km) [00:30:00]
	- 17:03:00 SS7 Ponte de Lima 2 (27.54km) [00:03:00]
	- 18:54:00 TC8 Porto - Rua Saraiva de Carvalho (99.27km) [01:51:00]
	- 19:03:00 SS8 Porto Street Stage1 (1.95km) [00:09:00]
	- 19:25:00 TC9 Porto - Calcada Vandoma (1.38km) [00:22:00]
	- 19:28:00 SS9 Porto Street Stage2 (1.95km) [00:03:00]
	- 20:00:00 TC9A Technical Zone IN (26.11km) [00:32:00]
	- 20:10:00 

In [516]:
sectionControls['code']

59    TC19B
60     TC20
61     SS20
62    TC20A
63    TC20B
64    TC20C
65    TC20D
Name: code, dtype: object

In [517]:
sectionControls#[sectionControls['code']]

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,roundingPolicy,timeZoneOffset,itinerarySectionId,section,order,mylocaltime
59,Sunday 20th May,TC19B,1916,None,NaN,31,2018-05-20 10:55:00,2018-05-20 10:55:00,Fafe - Regroup OUT,-1,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 11:55:00
60,Sunday 20th May,TC20,1953,All,8.42,31,2018-05-20 11:15:00,2018-05-20 11:15:00,Ruivaes,464,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 12:15:00
61,Sunday 20th May,SS20,1969,None,11.18,31,2018-05-20 11:18:00,2018-05-20 11:18:00,Fafe 2 (Power Stage),464,...,Minute,StageStart,82,188,RoundToClosestMinute,60,188,Section 7,7,2018-05-20 12:18:00
62,Sunday 20th May,TC20A,1917,All,85.97,31,2018-05-20 12:47:00,2018-05-20 12:47:00,Technical Zone IN,-1,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 13:47:00
63,Sunday 20th May,TC20B,1921,All,NaN,31,2018-05-20 12:50:00,2018-05-20 12:50:00,Technical Zone OUT - Service IN,-1,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 13:50:00
64,Sunday 20th May,TC20C,1899,All,NaN,31,2018-05-20 13:04:00,2018-05-20 13:04:00,Service OUT,-1,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 14:04:00
65,Sunday 20th May,TC20D,1897,All,7.75,31,2018-05-20 13:20:00,2018-05-20 13:20:00,Parc Ferme IN - FINISH (Matosinhos),-1,...,Minute,TimeControl,82,188,NoRounding,60,188,Section 7,7,2018-05-20 14:20:00


In [518]:
l

1